In [ ]:
!pip install "laspy[lazrs,laszip]"
!pip install rvt-py
!pip install rasterio

In [ ]:
import laspy
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rvt.default
import rvt.blend
import rasterio
from rasterio.transform import from_origin
import geopandas as gpd
import os
import pyproj

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

# Tile creation function definition

In [ ]:
def compute_slope(df_elevation, width, height):
    dem_arr=df_elevation['z'].to_numpy().reshape(width,height)
    default = rvt.default.DefaultValues()
    default.slp_output_units = "degree"
    slope_arr = default.get_slope(dem_arr=dem_arr, resolution_x=SAMPLING_IN_M, resolution_y=SAMPLING_IN_M)
#   plt.imshow(slope_arr, cmap='gray_r')
    slope_arr = np.clip(slope_arr/(55),0,1)
    #slope_arr = (slope_arr-np.amin(slope_arr))/(np.amax(slope_arr)-np.amin(slope_arr))
#   print(f'Values of slope are between {slope_arr.min()} and {slope_arr.max()}')
    return slope_arr

In [ ]:
def compute_svf_opns(df_elevation, width, height):
    dem_arr=df_elevation['z'].to_numpy().reshape(width,height)
    default = rvt.default.DefaultValues()
    svf_n_dir = 16  # number of directions
    svf_r_max = int(5/SAMPLING_IN_M) # max search radius in pixels
    svf_noise = 0  # level of noise remove (0-don't remove, 1-low, 2-med, 3-high)

    svf_opns_dict = default.get_sky_view_factor(dem_arr=dem_arr, resolution=SAMPLING_IN_M,
                                                    compute_svf=True, compute_asvf=False, compute_opns=True)

    svf_arr = svf_opns_dict["svf"]
    #svf_arr = np.clip((svf_arr-0.65)/0.35,0,1)
    svf_arr = (svf_arr-np.amin(svf_arr))/(np.amax(svf_arr)-np.amin(svf_arr))
    #plt.imshow(svf_arr, cmap='gray')

    #print(f'Skyview factor values are between {svf_arr.min()} and {svf_arr.max()}')

    opns_arr = svf_opns_dict["opns"]
    opns_arr = np.clip((opns_arr-60)/35,0,1)
    #opns_arr = (opns_arr-np.amin(opns_arr))/(np.amax(opns_arr)-np.amin(opns_arr))
    #plt.imshow(opns_arr, cmap='gray')

    #print(f'Positive openness are between {opns_arr.min()} and {opns_arr.max()}')

    return svf_arr, opns_arr

In [ ]:
def display_VAT_HS(df_elevation,width,height):
    slope_arr = 1 - compute_slope(df_elevation,width,height)
    svf_arr, opns_arr = compute_svf_opns(df_elevation,width,height)
    VAT_HS = np.dstack((slope_arr,opns_arr, svf_arr))
    return VAT_HS

In [ ]:
def write_VAT_HS_tiff(df_elevation,width,height):
    slope_arr = 1 - compute_slope(df_elevation,width,height)
    svf_arr, opns_arr = compute_svf_opns(df_elevation,width,height)
    VAT_HS = np.dstack((slope_arr,opns_arr, svf_arr))

    return VAT_HS